In [ ]:
'''
@Time    :   2025/03/04 18:20:30
@Author  :   Ran Zhang
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive&#8203;:contentReference[oaicite:1]{index=1}

# Configure the image folder path (update as needed)
image_folder = "/content/drive/MyDrive/BA820-Unsupervised ML/BA 820 Team 6/milestone 2/MarketingCharts"  # Path to your chart images folder

# Retrieve list of image files in the folder
import glob, os
image_paths = glob.glob(os.path.join(image_folder, "*.*"))  # e.g., "*.png" for PNG files
print(f"Found {len(image_paths)} images in the folder.")
for path in image_paths:
    print(os.path.basename(path))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1 images in the folder.
line-sale.png


In [ ]:
from google.colab import userdata

In [ ]:
import openai

# Set your API keys/tokens (replace with your actual keys)
openai.api_key = userdata.get('openai')  # OpenAI API key for ChatGPT
HF_API_TOKEN = userdata.get('BA820team6')  # if using Hugging Face Inference API for Qwen/Florence

# Define model endpoints or names
chatgpt_model = "gpt-4o-mini"  # using GPT-4 (assumed to have vision capabilities)
# qwen_model_id = "Qwen/QwenVL"  # example model name or endpoint for Qwen-VL
florence_model_id = "microsoft/Florence-2-large"  # example model name or endpoint for Florence-2

# (If using Hugging Face's pipeline or API, the model_id will be used there. If using custom API, set the URL.)


In [ ]:
# # Define the prompt template for chart analysis
# persona_note = ("Assume the user is a U.S.-based small business owner with at most some college education. "
#                 "Use plain, easy-to-understand English and avoid technical jargon.")
# task_prompt = (
#     "Analyze the marketing chart image and respond with two sections:\n"
#     "Chart Description (max 50 words): Identify the chart type, explain its content, and summarize key data points in simple terms.\n"
#     "Insights & Actions (max 50 words): Provide actionable insights, including next steps or takeaways relevant to the user.\n"
# )
# # Combine into a final prompt for models that need a single prompt string
# full_prompt_text = persona_note + "\n" + task_prompt


In [ ]:
full_prompt_text = "Describe the business insights and actions based on the chart. Assume the user is a U.S.-based small business owner with at most some college education. Use plain, easy-to-understand English and avoid technical jargon."

###Qwen

In [ ]:
!pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 57.7 MB/s eta 0:00:00


In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

In [ ]:
import torch
from PIL import Image

# Load Qwen2-VL-7B-Instruct model and processor
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype="auto",        # automatic mixed precision (fp16 if GPU available)
    device_map="auto"          # automatically use available GPU/CPU devices
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
import glob, os
from PIL import Image

# Retrieve list of image files in the folder
image_paths = glob.glob(os.path.join(image_folder, "*.*"))  # adjust pattern if needed
print(f"Found {len(image_paths)} images in the folder.")
for path in image_paths:
    print(os.path.basename(path))

# # Example for processing one image
# # (Within your processing loop, use each image file path)
# image_path = image_paths[0]  # For example, process the first image
# prompt_text = full_prompt_text

# # Open the image using PIL
# image = Image.open(image_path)

# # Prepare the conversation with image + prompt for the user role
# conversation = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "image"},  # placeholder for the image (the processor will attach the actual image)
#             {"type": "text", "text": prompt_text}
#         ]
#     }
# ]

# # Now, use the conversation along with the image in your Qwen inference function

Found 1 images in the folder.
line-sale.png


In [ ]:
def extract_insights_from_image(image_path, prompt_text):
    try:
        from PIL import Image
        image = Image.open(image_path)
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": prompt_text}
                ]
            }
        ]

        # Set or pass the chat template as needed:
        processor.chat_template = full_prompt_text
        # text_input = processor.apply_chat_template(conversation, add_generation_prompt=True)
        # text_input = processor.apply_chat_template(conversation, chat_template=full_prompt_text, add_generation_prompt=True)


        # Prepare inputs with the processor
        inputs = processor(text=[text_input], images=[image], return_tensors="pt", padding=True)

        # Move inputs to the device.
        # Only cast pixel_values to float16; keep input_ids as integers.
        for key, tensor in inputs.items():
            if key == "pixel_values":
                inputs[key] = tensor.to(model.device, dtype=torch.float16)
            else:
                inputs[key] = tensor.to(model.device)

        # Generate output tokens
        output_ids = model.generate(**inputs, max_new_tokens=128)
        prompt_len = inputs.input_ids.shape[1]
        gen_tokens = output_ids[:, prompt_len:]
        generated_texts = processor.batch_decode(gen_tokens, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        insight_text = generated_texts[0].strip() if generated_texts else ""
        if not insight_text:
            insight_text = "No insights were generated for this image."
        return insight_text
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return "Error: unable to process image"


###florence

In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForCausalLM

# Set the device and torch dtype
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32

# Define the Florence model ID (adjust if needed)
florence_model_id = "microsoft/Florence-2-large"

# Load the processor and model for Florence
processor = AutoProcessor.from_pretrained(florence_model_id)
model = AutoModelForCausalLM.from_pretrained(florence_model_id, torch_dtype=torch_dtype).to(device)



preprocessor_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

The repository for microsoft/Florence-2-large contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Florence-2-large.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


processing_florence2.py:   0%|          | 0.00/48.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

...

The repository for microsoft/Florence-2-large contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Florence-2-large.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
The repository for microsoft/Florence-2-large contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Florence-2-large.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

In [ ]:
def run_example(task_prompt, image, text_input=None):
    if text_input is None:
        prompt = full_prompt_text

    else:
        prompt = full_prompt_text

    image = image.convert('RGB')
    inputs = processor(text=prompt, images=image, return_tensors="pt")

    # Cast only pixel_values to float16; leave input_ids as integers
    for k, v in inputs.items():
        if k == "pixel_values":
            inputs[k] = v.to(device, dtype=torch.float16)
        else:
            inputs[k] = v.to(device)

    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        early_stopping=False,
        do_sample=False,
        num_beams=3,
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer


###output

In [ ]:
persona_note = "Assume the user is a U.S.-based small business owner with at most some college education. Use plain, easy-to-understand English and avoid technical jargon."
task_prompt = "Describe the business insights and actions based on the chart."

In [ ]:
results = []  # to accumulate results for each image

for img_path in image_paths:
    image_name = os.path.basename(img_path)
    print(f"\nProcessing image: {image_name}")

    # 5.1 ChatGPT analysis
    chat_messages = [
    {"role": "system", "content": persona_note},
    {"role": "user", "content": task_prompt}
    ]
    # Use the updated OpenAI client for ChatGPT
    from openai import OpenAI
    client = OpenAI(api_key=openai.api_key)  # Make sure openai.api_key is set

    chat_response = client.chat.completions.create(
      model=chatgpt_model,
      messages=chat_messages
    )
    chat_text = chat_response.choices[0].message.content.strip()
    print("ChatGPT Output:\n", chat_text)

    # 5.2 Qwen analysis using local inference (replacing the API-based call)
    processor.chat_template = full_prompt_text
    qwen_text = extract_insights_from_image(img_path, full_prompt_text)
    print("Qwen Output:\n", qwen_text)

    # 5.3 Florence analysis using local inference
    # If you have a similar function for Florence, replace accordingly.
    # Here, we'll assume you use the same function for demonstration:
    image = Image.open(img_path)
    florence_text = run_example(task_prompt, image)  # you can append text if needed
    print("Florence Output:\n", florence_text)

    # Save the insights from each model for this image
    results.append({
        "image": image_name,
        "ChatGPT_Insight": chat_text,
        "Qwen_Insight": qwen_text,
        "Florence_Insight": florence_text
    })

    # Optionally, pause for manual review before moving to evaluation
    _ = input("Press Enter to continue to evaluation for this image...")



Processing image: line-sale.png
ChatGPT Output:
 **Chart Description:**  
The chart is a bar graph showing monthly sales from January to December. It highlights peak sales in December and the lowest in February. Overall, there’s a noticeable increase during the holiday season.

**Insights & Actions:**  
Focus marketing efforts on December leading up to the holidays. Consider promotions in February to boost sales when they are typically low. Examine what worked well in December to replicate that success.
Error processing /content/drive/MyDrive/BA820-Unsupervised ML/BA 820 Team 6/milestone 2/MarketingCharts/line-sale.png: name 'text_input' is not defined
Qwen Output:
 Error: unable to process image
Florence Output:
 {'Analyze the marketing chart image and respond with two sections:\nChart Description (max 50 words): Identify the chart type, explain its content, and summarize key data points in simple terms.\nInsights & Actions (max 50 words): Provide actionable insights, including next 

###evaluation

In [ ]:
# Extend the results dicts with evaluation scores
for result in results:
    image_name = result["image"]
    print(f"\nEvaluating insights for image: {image_name}")
    chat_insight = result["ChatGPT_Insight"]
    qwen_insight = result["Qwen_Insight"]
    florence_insight = result["Florence_Insight"]

    # Prepare prompts for each evaluation
    # 6.1 ChatGPT evaluates Qwen
    eval_prompt_cq = (
        f"Chart description: {chat_insight.split('Insights & Actions')[0].strip()}\n"  # using ChatGPT's description as context
        f"Insight to evaluate: {qwen_insight}\n"
        "Rate this insight's Relevance, Coherence, and Actionability on a 1-5 scale.\n"
        "Provide output as JSON: {\"Relevance\":#, \"Coherence\":#, \"Actionability\":#}."
    )
    eval_cq_resp = openai.ChatCompletion.create(
        model=chatgpt_model,
        messages=[{"role": "user", "content": eval_prompt_cq}]
    )
    eval_cq = eval_cq_resp['choices'][0]['message']['content']

    # 6.2 ChatGPT evaluates Florence
    eval_prompt_cf = (
        f"Chart description: {chat_insight.split('Insights & Actions')[0].strip()}\n"
        f"Insight to evaluate: {florence_insight}\n"
        "Rate this insight on Relevance, Coherence, Actionability (1-5 each) and respond in JSON."
    )
    eval_cf_resp = openai.ChatCompletion.create(
        model=chatgpt_model,
        messages=[{"role": "user", "content": eval_prompt_cf}]
    )
    eval_cf = eval_cf_resp['choices'][0]['message']['content']

    # 6.3 Qwen evaluates ChatGPT
    qwen_eval_input = f"Chart: {qwen_insight.split('Insights & Actions')[0].strip()}\nInsight: {chat_insight}\nRelevance? Coherence? Actionability? (1-5 each)."
    qwen_eval_resp = call_qwen_inference(image_folder + "/" + image_name, qwen_eval_input)
    eval_qc = qwen_eval_resp if isinstance(qwen_eval_resp, str) else qwen_eval_resp.get('generated_text', '')

    # 6.4 Qwen evaluates Florence
    qwen_eval_input2 = f"Chart: {qwen_insight.split('Insights & Actions')[0].strip()}\nInsight: {florence_insight}\nRelevance? Coherence? Actionability? (1-5 each)."
    qwen_eval_resp2 = call_qwen_inference(image_folder + "/" + image_name, qwen_eval_input2)
    eval_qf = qwen_eval_resp2 if isinstance(qwen_eval_resp2, str) else qwen_eval_resp2.get('generated_text', '')

    # 6.5 Florence evaluates ChatGPT
    florence_eval_input = f"Chart: {florence_insight.split('Insights & Actions')[0].strip()}\nInsight: {chat_insight}\nRelevance? Coherence? Actionability? (1-5 each)."
    florence_eval_resp = call_florence_inference(image_folder + "/" + image_name, florence_eval_input)
    eval_fc = florence_eval_resp if isinstance(florence_eval_resp, str) else florence_eval_resp.get('generated_text', '')

    # 6.6 Florence evaluates Qwen
    florence_eval_input2 = f"Chart: {florence_insight.split('Insights & Actions')[0].strip()}\nInsight: {qwen_insight}\nRelevance? Coherence? Actionability? (1-5 each)."
    florence_eval_resp2 = call_florence_inference(image_folder + "/" + image_name, florence_eval_input2)
    eval_fq = florence_eval_resp2 if isinstance(florence_eval_resp2, str) else florence_eval_resp2.get('generated_text', '')

    # Parse and store scores (assuming JSON or structured output)
    # Simplified parsing: extracting digits
    import re
    def parse_scores(text):
        nums = re.findall(r'\d', text)
        return nums[0] if nums else None, nums[1] if len(nums)>1 else None, nums[2] if len(nums)>2 else None

    result["ChatGPT_eval_Qwen"] = parse_scores(eval_cq)
    result["ChatGPT_eval_Florence"] = parse_scores(eval_cf)
    result["Qwen_eval_ChatGPT"] = parse_scores(eval_qc)
    result["Qwen_eval_Florence"] = parse_scores(eval_qf)
    result["Florence_eval_ChatGPT"] = parse_scores(eval_fc)
    result["Florence_eval_Qwen"] = parse_scores(eval_fq)

    print(f"ChatGPT->Qwen eval: {result['ChatGPT_eval_Qwen']}, ChatGPT->Florence eval: {result['ChatGPT_eval_Florence']}")
    # (Similarly print or inspect other evaluation results as needed)


###export

In [ ]:
import pandas as pd

# Flatten the score tuples into separate columns for clarity
for res in results:
    if isinstance(res["ChatGPT_eval_Qwen"], tuple):
        res["ChatGPT_eval_Qwen_Relevance"] = res["ChatGPT_eval_Qwen"][0]
        res["ChatGPT_eval_Qwen_Coherence"] = res["ChatGPT_eval_Qwen"][1]
        res["ChatGPT_eval_Qwen_Actionability"] = res["ChatGPT_eval_Qwen"][2]
        # ... repeat for each evaluation field ...
        # (It's a bit verbose; consider looping over evaluator-target pairs to automate)
    # Remove the tuple fields if not needed
    # res.pop("ChatGPT_eval_Qwen", None)  # etc.

# Create DataFrame and save
df = pd.DataFrame(results)
output_csv = os.path.join(image_folder, "chart_insights_evaluation.csv")
df.to_csv(output_csv, index=False)
print(f"Results saved to {output_csv}")
